In [1]:
import os

In [2]:
%pwd

'c:\\Users\\quamr\\OneDrive\\Desktop\\project\\kidney\\research'

In [3]:
os.chdir("C:\\Users\\quamr\\OneDrive\\Desktop\\project\\kidney")

In [4]:
%pwd

'C:\\Users\\quamr\\OneDrive\\Desktop\\project\\kidney'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    training_data:str
    trained_model_path: Path
    updated_base_model_path: Path
    params_epochs: int
    params_batch_size: int
    params_image_size: list
    params_is_augmentation: bool

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [10]:
class ConfigurationManager:
  def __init__(
      self,
      config_filepath:Path=CONFIG_FILE_PATH,
      params_filepath:Path=PARAMS_FILE_PATH
  ):
    self.config = read_yaml(config_filepath)
    self.params = read_yaml(params_filepath)
    create_directories([self.config.artifacts_root])

  def get_training_config(self) -> TrainingConfig:
    training = self.config.training
    prepare_base_model = self.config.prepare_base_model
    params = self.params
    training_data = os.path.join(self.config.data_ingestion.unzip_dir, "C:\\Users\\quamr\\OneDrive\\Desktop\\project\\kidney\\artifacts\\data_ingestion\\content\\drive\\MyDrive\\CT-kidney-data")
    create_directories([training.root_dir])

    training_config = TrainingConfig(
      root_dir = Path(training.root_dir),
      training_data = training_data,
      trained_model_path = Path(training.trained_model_path),
      updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
      params_batch_size=params.BATCH_SIZE,
      params_image_size=params.IMAGE_SIZE,
      params_epochs=params.EPOCHS,
      params_is_augmentation=params.AUGMENTATION  
    )
    return training_config

In [11]:
import os 
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time


In [14]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1.0 / 255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=tuple(self.config.params_image_size[:2]),
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
            class_mode="categorical"
        )

        # Validation generator (NO augmentation)
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        # Training generator (WITH augmentation)
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=20,
                zoom_range=0.2,
                width_shift_range=0.1,
                height_shift_range=0.1,
                shear_range=0.1,
                horizontal_flip=True,
                brightness_range=[0.8, 1.2],
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = math.ceil(
            self.train_generator.samples / self.train_generator.batch_size
        )
        self.validation_steps = math.ceil(
            self.valid_generator.samples / self.valid_generator.batch_size
        )

        callbacks = [
            # Save best model
            tf.keras.callbacks.ModelCheckpoint(
                filepath=self.config.trained_model_path,
                save_best_only=True,
                monitor="val_loss",
                verbose=1
            ),

            # Stop training when no improvement
            tf.keras.callbacks.EarlyStopping(
                monitor="val_loss",
                patience=7,
                restore_best_weights=True,
                verbose=1
            ),

            # Reduce LR on plateau
            tf.keras.callbacks.ReduceLROnPlateau(
                monitor="val_loss",
                factor=0.3,
                patience=4,
                min_lr=1e-6,
                verbose=1
            ),

            # Save logs to CSV
            tf.keras.callbacks.CSVLogger(
                filename=str(self.config.root_dir / "training_logs.csv"),
                append=True
            ),

            # TensorBoard visualization
            tf.keras.callbacks.TensorBoard(
                log_dir=str(self.config.root_dir / "tensorboard_logs"),
                histogram_freq=1
            )
        ]

        self.model.fit(
            self.train_generator,
            validation_data=self.valid_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            callbacks=callbacks
        )


In [13]:
try:
  config = ConfigurationManager()
  training_config = config.get_training_config()
  training = Training(config=training_config)
  training.get_base_model()
  training.train_valid_generator()
  training.train()
except Exception as e:
  raise e

[2026-01-04 16:00:41,618:INFO:common:yaml file: config\config.yaml loaded successfully]
[2026-01-04 16:00:41,664:INFO:common:yaml file: params.yaml loaded successfully]
[2026-01-04 16:00:41,672:INFO:common:created directory at: artifacts]
[2026-01-04 16:00:41,675:INFO:common:created directory at: artifacts/training]
Found 2489 images belonging to 4 classes.
Found 9969 images belonging to 4 classes.
624/624 [==============================] - ETA: 0s - loss: 11.8645 - accuracy: 0.4642
Epoch 1: val_loss improved from inf to 12.86073, saving model to artifacts\training\model.h5
624/624 [==============================] - 2194s 4s/step - loss: 11.8645 - accuracy: 0.4642 - val_loss: 12.8607 - val_accuracy: 0.3969 - lr: 0.0100


In [ ]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/quamrl-hoda/diseaseClassification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "quamrl-hoda"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "10bd4471835788590657bbf53e006492e901bc34"